# Part C - Increase the number of epochs

Repeat Part B but use 100 epochs this time for training.<br>
<br>
How does the mean of the mean squared errors compare to that from Step B?

In [1]:
#import libraries
import pandas as pd
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense, Input

from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter('ignore', FutureWarning)

2025-01-21 19:32:03.554688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737484323.576693  986003 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737484323.583323  986003 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 19:32:03.618932: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Import and check data

In [2]:
# download the data and read it into a pandas dataframe.
concrete_data = pd.read_csv("concrete_data.csv")

concrete_data.shape

(1030, 9)

In [3]:
# Already done in part A
#concrete_data.head()
#concrete_data.tail()
#concrete_data.isnull().sum()

#### Split data into predictors and target

In [4]:
predictors = concrete_data[concrete_data.columns[concrete_data.columns != 'Strength']] 
target = concrete_data['Strength'] 
print(f"predictors shape: {predictors.shape}")
print(f"target shape: {target.shape}")

predictors shape: (1030, 8)
target shape: (1030,)


#### Normalize the data

In [5]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


### Build the neural network

In [6]:
# define same regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(8,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')   
    return model

### Repeat 50 times now and show mean and standard deviation of the results

This time training the model 100 times

In [7]:
model = regression_model()
mean_squared_errors = []

2025-01-21 19:32:06.855343: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
%%time

for i in range(50):
    # i) split data
    x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=9)
    ## ii) fit model
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, verbose=0)
    # iii) evaluate model
    mse = model.evaluate(x_test, y_test, verbose=0)
    mean_squared_errors.append(mse)

CPU times: user 12min 5s, sys: 40.3 s, total: 12min 45s
Wall time: 10min 44s


Calculate mean and std. deviation

In [9]:
#Calculate the mean and standard deviation of the mean squared errors
mse_mean = np.mean(mean_squared_errors)
mse_std = np.std(mean_squared_errors)

print(f"Mean of calculated mse: {mse_mean}")
print(f"Standard deviation of calculated mse: {mse_std}")

Mean of calculated mse: 57.55678237915039
Standard deviation of calculated mse: 10.578683935862834


How does the mean of the mean squared errors compare to that from Step B?

First execution:<br>
Mean of calculated mse: 52.08996025085449  <br>
Std. deviation of calculated mse: 15.724933259168857 <br>
CPU times: user 12min 9s, sys: 38.7 s, total: 12min 48s <br>
Wall time: 10min 46s <br>
<br>
Second execution:<br>
Mean of calculated mse:  57.55678237915039 <br>
Std. deviation of calculated mse: 10.578683935862834  <br>
CPU times: user 12min 5s, sys: 40.3 s, total: 12min 45s <br>
Wall time: 10min 44s

The results are better than in part B. (But at the cost of a much higher execution time because of epoch = 100)